In [ ]:
import os
from pymongo import MongoClient
import numpy as np
import utils.const as const
import pandas as pd
from ast import literal_eval
import re

In [ ]:
db=const.grpId
collection=const.PostCollection
fileName='word_vector.csv'

post_out='text_post.csv'
comment_out='text_comment.csv'
subComment_out='text_subComment.csv'

outputLen=100

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
class Node:
    def __init__(self,length=outputLen):
        self.arr=np.zeros((length),dtype='float32')
        self.count=0
  
    def insert(self,arr2):
        arrT=self.arr.astype('float64')*self.count + arr2
        self.count+=1
        arrT/=self.count

        self.arr=arrT.astype('float32')
  
    def getString(self):
        ans=''
        for i in self.arr:
            ans+=','+str(i)

        return ans

In [ ]:
def handleStupidity(s):
    s=re.sub('\[ ','[',s)
    return np.array(literal_eval(re.sub('[ \n]+',',',s))).astype('float32')

In [ ]:
df = pd.read_csv(fileName)
df.vector = df.vector.apply(handleStupidity)
df.head()

In [ ]:
wordVec={}

for i in range(len(df.index)):
    wordVec[df['word'][i]]=df['vector'][i]

del df

In [ ]:
fp = open(post_out,'w')

for post in myCollect.find():
    node= Node()
    if 'message' in post and isinstance(post['message'],list):
        for k in post['message']:
            if k in wordVec:
                node.insert(wordVec[k])
    
    fp.write(post['_id'])
    fp.write(node.getString())
    fp.write('\n')

fp.close()

In [ ]:
fp = open(comment_out,'w')

for post in myCollect.find():
    if 'comments' in post:
        for comment in post['comments']:
            node= Node()
            if 'message' in comment and isinstance(comment['message'],list):
                for k in comment['message']:
                    if k in wordVec:
                        node.insert(wordVec[k])
            
            fp.write(post['_id'])
            fp.write(','+comment['_id'])
            fp.write(node.getString())
            fp.write('\n')

fp.close()

In [ ]:
fp = open(subComment_out,'w')

for post in myCollect.find():
    if 'comments' in post:
        for comment in post['comments']:
            if 'subComments' in comment:
                for subComment in comment['subComments']:
                    node= Node()
                    if 'message' in subComment and isinstance(subComment['message'],list):
                        for k in subComment['message']:
                            if k in wordVec:
                                node.insert(wordVec[k])

                    fp.write(post['_id'])
                    fp.write(','+comment['_id'])
                    fp.write(','+subComment['_id'])
                    fp.write(node.getString())
                    fp.write('\n')

fp.close()